# Generate pseudo-words using a Markov chain 

- We will read in a dataset of English words, with pronunciations, calculate phoneme transition probabilities from the data, and then use those phoneme transition probabilities to generate pseudo-words that follow the phonotactic probabilities of English.
- This is essentially a simple Markov chain.  Every phoneme is a state - specifically, being in the state corresponding to phoneme $p$ means that we have just seen phoneme $p$.  Transition probabilities from state $p$ to some other state $q$ are determined from bigrams in the data.  How?
- A bigram is a pair of consecutive symbolic units - in this case phonemes.  Thus transition probabilities such as $P(p_2|p_1)$ represent the probability of moving from the state that denotes our having just seen phoneme $p_1$ to the state that denotes our having just seen phoneme $p_2$;  this probability is determined by the relative frequency of encountering $p_2$ in the data, given that we have just seen $p_1$.
- This notebook again uses [WikiPron](https://github.com/kylebgorman/wikipron), but this time we consider US pronunciation for English.  Analogous data for many other languages, in the same format, is available [here](https://github.com/kylebgorman/wikipron/tree/master/data), which you can download and use for extensions.

In [1]:
import re  # not necessary here
import pandas as pd
import numpy as np
import unicodedata
import random  # useful for sampling from a distribution

### Read in data

And format it so that pulling out phoneme transitions will be easy.

In [2]:
# read in english words.  
df = pd.read_csv("eng_us_phonemic.tsv", sep='\t', names=['Word', 'Pron'])
df.Pron = df.Pron.str.split()  # turn space-separated strings into lists of phonemes
df

,Word,Pron
0,$deity,"[d, e, ɪ, ɪ, t, i]"
1,$deity,"[d, iː, ɪ, t, i]"
2,'cause,"[k, ʌ, z]"
3,'dswounds,"[d, z, w, u, n, d, z]"
4,'em,[m̩]
...,...,...
53394,œconomy,"[ɪː, k, ɒ, n, ɒ, m, i]"
53395,œneus,"[ɛ, n, j, uː, s]"
53396,ǃkung,"[k, u, ŋ]"
53397,ǃkung,"[k, ʊ, ŋ]"


In [3]:
# sanity check
# find the US pronunciation of the word "aardvark"
df[df.Word=='aardvark']

,Word,Pron
67,aardvark,"[ɑ, ɹ, d, v, ɑ, ɹ, k]"


In [4]:
# sanity check - and a chance to introduce the very useful pandas "sample" routine.
# find the US pronunciation of several other words.
df.sample(n=25)  # pick 25 random samples

,Word,Pron
28223,landform,"[l, æ, n, d, f, ɔ, ɹ, m]"
43374,sheela,"[ʃ, iː, l, ə]"
43925,sines,"[s, a, ɪ, n, z]"
22364,greeting,"[ɡ, ɹ, iː, t, ɪ, ŋ]"
3754,anvil,"[æ, n, v, ɪ, l]"
46463,supersession,"[s, u, p, ɚ, s, ɛ, ʃ, ə, n]"
38369,principled,"[p, ɹ, ɪ, n, s, ə, p, ə, l, d]"
5204,awaked,"[ə, w, e, ɪ, k, t]"
31064,memoir,"[m, ɛ, m, w, ɑ, ɹ]"
27742,kisses,"[k, ɪ, s, ɪ, z]"


In [5]:
# now add START and STOP pseudo-phonemes at beginning and end of each word.
# a chance to introduce the very useful pandas "apply" routine.
df.Pron = df.Pron.apply(lambda x: ['START'] + x + ['STOP'])
df

,Word,Pron
0,$deity,"[START, d, e, ɪ, ɪ, t, i, STOP]"
1,$deity,"[START, d, iː, ɪ, t, i, STOP]"
2,'cause,"[START, k, ʌ, z, STOP]"
3,'dswounds,"[START, d, z, w, u, n, d, z, STOP]"
4,'em,"[START, m̩, STOP]"
...,...,...
53394,œconomy,"[START, ɪː, k, ɒ, n, ɒ, m, i, STOP]"
53395,œneus,"[START, ɛ, n, j, uː, s, STOP]"
53396,ǃkung,"[START, k, u, ŋ, STOP]"
53397,ǃkung,"[START, k, ʊ, ŋ, STOP]"


### Get bigram conditional probabilities for each pair of phonemes

In [6]:
# start by getting the set of all phonemes in the dataset.
# we will have one state for each phoneme.
phonemes = set([p for l in df.Pron for p in l]) # for every list in df.pron; for every phoneme in that list
print(len(phonemes))
print(phonemes)

186
{'ʊ̂', 'ɾ̃', 'STOP', 'æ', 't͡s', 'ʉː', 'ɚː', 'eː', 'ú', 'iː', 'pː', 'ē', 'ɛ͡ɪ', 'ɡː', 'ɑ̃', 'ʃʰ', 'bᵊ', 'ɔ̃', 'i', 'ɔ', 'nˡ', 'ɛ̝', 'ʍ', 'ːs', 'ɹ̩', 'f', 'u', 'k̚', 'ɣ', 'ĭ', 'nː', 'îː', 'dʰ', 'ʊ̩', 'ʊ̯', 'ɪ̈', 'oˀ', 'ç', 'ɹʷ', 'ɹː', 'õ', 'ɫ̩', 'o̞', 'ɫ', 't͡ɕ', 'fː', 'ʊː', 'e͡ɪ', 'ɒː', 'ɻ', 'ä', 'ɑː', 't͡ʃ', 'y', 'm̩', 'ʒ', 'd͡ʒ', 'ɑˑ', 'ɪʲ', 'e͜ɚ', 'ʀ', 'a', 'ɜː', 'ɐ', 'k', 'kʰ', 'v', 't͡ʃʰ', 'r̩', 'ã', '↓', 'ʊ̆', 'd̪', 'χ', 'ŋ', 'ɘ', 'ɒ', 't', 'θ', 'lʲ', 'ɪˑ', 'ɱ', 'ɪ̪', 'æ̃', 'ɛː', 'ʊ̪', 'e', 'o͡ʊ', 'ɲ', 'p̬', '˞', 'b', 'tʰ', 'ɾ', 'ʔ', 'ĩ', 'ɔ͡ɪ', 'ɖ', 'əˑ', '~', 'ɭ', 'ɵ', 'ə̯', 'sː', 'ɐː', 'n', 'ʙ̩ː', 'ʌ', 'n̩', 'ɯ', 'w', 'tᵊ', 'ɜ', 'oː', 'ŭ̥', 'ɪ', 'ɚ', 'ɪ̯', 'ø', 'ɝː', 'ɝ', 'c', 'ɹ', 'ĭ̥', 'bʲ', 'l͡s', 'l', 'm̩ː', 'ṽ', 'ə', 'ɛ̃', 'ɒ̃', 'ɬ', 'ɜ̃', 'ʌ̈', 't͡ʃ̩̩', 'ɡ', 'äː', 'əʳ', 'dʲ', 'ʈ', 'tː', 'ð', 'kᵊ', 'ɛ͡ɹ', 'ɦ', 'ɽ', 'd', 'q', 'ɔː', 'ɕː', 'æː', 'r', 'ɛˑ', 'j', 't̬', 'ʊ', 'START', 'ʃː', 'ɵ̞', 'ʃ', 'œ', 'h', 'p', 't̬ᵊ', 'm', 'əː', 'ə̆', 'ɛ', 'ɑ', 's', 'ʉ', 'pʰ', 'mː', 'ô'

Now get simple phoneme transition **counts** for every pair of phonemes.  Later, we will turn these into probabilities.

We will use [Laplace (add-one) smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), which means that every phoneme pair gets a "freebie" count of 1 to start with, before actually seeing any data.  Ask me why we might want to do such a thing.

In [7]:
# ptc = phoneme transition counts.
# this structure is a dict.
# its key is a tuple of 2 phonemes: (p1, p2), and the corresponding value is the count for those 2 phonemes.
# here, we start by initializing each count with 1: add-one smoothing.
# MODIFY THIS NEXT PART TO INITIALIZE ALL COUNTS TO 1.
ptc = {}  # YOUR CODE GOES HERE.

for phoneme1 in phonemes:
    for phoneme2 in phonemes:
        ptc[(phoneme1, phoneme2)] = 1

print(len(ptc)) # sanity check
        
ptc

34596


{('ʊ̂', 'ʊ̂'): 1,
 ('ʊ̂', 'ɾ̃'): 1,
 ('ʊ̂', 'STOP'): 1,
 ('ʊ̂', 'æ'): 1,
 ('ʊ̂', 't͡s'): 1,
 ('ʊ̂', 'ʉː'): 1,
 ('ʊ̂', 'ɚː'): 1,
 ('ʊ̂', 'eː'): 1,
 ('ʊ̂', 'ú'): 1,
 ('ʊ̂', 'iː'): 1,
 ('ʊ̂', 'pː'): 1,
 ('ʊ̂', 'ē'): 1,
 ('ʊ̂', 'ɛ͡ɪ'): 1,
 ('ʊ̂', 'ɡː'): 1,
 ('ʊ̂', 'ɑ̃'): 1,
 ('ʊ̂', 'ʃʰ'): 1,
 ('ʊ̂', 'bᵊ'): 1,
 ('ʊ̂', 'ɔ̃'): 1,
 ('ʊ̂', 'i'): 1,
 ('ʊ̂', 'ɔ'): 1,
 ('ʊ̂', 'nˡ'): 1,
 ('ʊ̂', 'ɛ̝'): 1,
 ('ʊ̂', 'ʍ'): 1,
 ('ʊ̂', 'ːs'): 1,
 ('ʊ̂', 'ɹ̩'): 1,
 ('ʊ̂', 'f'): 1,
 ('ʊ̂', 'u'): 1,
 ('ʊ̂', 'k̚'): 1,
 ('ʊ̂', 'ɣ'): 1,
 ('ʊ̂', 'ĭ'): 1,
 ('ʊ̂', 'nː'): 1,
 ('ʊ̂', 'îː'): 1,
 ('ʊ̂', 'dʰ'): 1,
 ('ʊ̂', 'ʊ̩'): 1,
 ('ʊ̂', 'ʊ̯'): 1,
 ('ʊ̂', 'ɪ̈'): 1,
 ('ʊ̂', 'oˀ'): 1,
 ('ʊ̂', 'ç'): 1,
 ('ʊ̂', 'ɹʷ'): 1,
 ('ʊ̂', 'ɹː'): 1,
 ('ʊ̂', 'õ'): 1,
 ('ʊ̂', 'ɫ̩'): 1,
 ('ʊ̂', 'o̞'): 1,
 ('ʊ̂', 'ɫ'): 1,
 ('ʊ̂', 't͡ɕ'): 1,
 ('ʊ̂', 'fː'): 1,
 ('ʊ̂', 'ʊː'): 1,
 ('ʊ̂', 'e͡ɪ'): 1,
 ('ʊ̂', 'ɒː'): 1,
 ('ʊ̂', 'ɻ'): 1,
 ('ʊ̂', 'ä'): 1,
 ('ʊ̂', 'ɑː'): 1,
 ('ʊ̂', 't͡ʃ'): 1,
 ('ʊ̂', 'y'): 1,
 ('ʊ̂', 'm̩'): 1,
 ('ʊ̂', 'ʒ'): 1,
 (

In [8]:
# now add in the actual bigram counts
prons = df.Pron
for pron in prons:
    # YOUR CODE GOES HERE.
    # increment the ptc count for (p1,p2) each time you encounter the bigram "p1 p2" in the data.
    for i in range(len(pron) - 1):
        ptc[(pron[i], pron[i + 1])] += 1
    

# sanity-check some of the counts.
print("START d", ptc[('START','d')])
print("d START", ptc[('d','START')])
print("f ɹ", ptc[('f','ɹ')])
print("ɹ f", ptc[('ɹ','f')])
print("g ɹ", ptc[('ɡ','ɹ')])
print("ɹ g", ptc[('ɹ','ɡ')])
print("s l", ptc[('s','l')])
print("l s", ptc[('l','s')])

START d 3380
d START 1
f ɹ 500
ɹ f 153
g ɹ 875
ɹ g 125
s l 404
l s 135


Now transform the phoneme transition **counts** into phoneme transition **probabilities**.  

For every pair of phonemes $p_1, p_2$, we already have ptc($p_1$,$p_2$): the phoneme transition count from $p_1$ to $p_2$, i.e. the number of times we encountered the bigram $p_1 p_2$ in the data.  

From this, we want to obtain the transition **probability** $p(p_2|p_1)$, i.e. the probability of encountering $p_2$, given that we have just seen $p_1$.  Note the order of the two phonemes: our bigram is $p_1 p_2$, and we use the count for it (and other bigrams) to obtain $p(p_2|p_1)$.  

How?  This is a conditional probability, and following the Bishop chapter, or Jurafsky and Martin eqn 3.11, we may calculate it as:

\begin{equation}
p(p_2|p_1) = \frac{c(p_1 p_2)}{c(p_1)}
\end{equation}

where $c(\cdot)$ denotes count.  The numerator is simply the count for the bigram $p_1 p_2$.  The denominator is the count for $p_1$ whatever phoneme may follow it.  We have bigram counts, so we can easily obtain the denominator $c(p_1)$ as: 

\begin{equation}
c(p_1) = \sum_{q \in P} c(p_1 q)
\end{equation}

where $P$ is the set of all phonemes.

In [9]:
# data structure for transition probabilities is a dict of dicts.
# the outer dict is keyed by phoneme (state), and has value inner dict.
# the inner dict is also keyed by phoneme, and holds the probability of moving from the outer phoneme
# to the inner phoneme, i.e. p(inner|outer).
# this structure should make it easy to convert counts to probs, normalize etc.

tp = dict()  # transition probs, outer dict.
for p1 in phonemes:
    d = dict()  # inner dict
    # YOUR CODE GOES HERE.
    p1_total = 0
    for key in ptc.keys():
        p1_total += ptc[key] if key[0] == p1 else 0
    for p2 in phonemes:
        d[p2] = ptc[(p1, p2)] / p1_total
    
    tp[p1] = d

In [10]:
# sanity check.
# the conditional probabilities p(p2|p1) should add up to 1.0 when summing across p2, for each p1.
print("Sanity check: do the conditional probabilities conditioned on each phoneme sum to 1.0?")
for p1 in phonemes:
    total = sum(tp[p1].values())
    print("Sum of conditional probabilities conditioned on", p1, total)

Sanity check: do the conditional probabilities conditioned on each phoneme sum to 1.0?
Sum of conditional probabilities conditioned on ʊ̂ 1.0000000000000027
Sum of conditional probabilities conditioned on ɾ̃ 1.000000000000002
Sum of conditional probabilities conditioned on STOP 0.9999999999999992
Sum of conditional probabilities conditioned on æ 0.9999999999999991
Sum of conditional probabilities conditioned on t͡s 0.9999999999999959
Sum of conditional probabilities conditioned on ʉː 0.9999999999999959
Sum of conditional probabilities conditioned on ɚː 1.000000000000002
Sum of conditional probabilities conditioned on eː 0.9999999999999978
Sum of conditional probabilities conditioned on ú 1.0000000000000027
Sum of conditional probabilities conditioned on iː 1.000000000000001
Sum of conditional probabilities conditioned on pː 1.0000000000000049
Sum of conditional probabilities conditioned on ē 0.9999999999999959
Sum of conditional probabilities conditioned on ɛ͡ɪ 1.0000000000000027
Sum o

### Congratulations!  You have created a Markov chain.

Now use it to generate pseudo-words.

In [11]:
# specifically, let's generate 20 pseudo-words.
for i in range(20):
    # for each pseudo-word, we start by setting state to 'START'.
    state = 'START'
    print(state, end='')   # print state, but no newline.

    # now keep repeating this until you arrive at the 'STOP' state
    while (state != 'STOP'):
        next_state_options = list(tp[state].keys())
        next_state_probs = list(tp[state].values())
        new_state = np.random.choice(next_state_options, p=next_state_probs)
        print(" ", new_state, end='')
        state = new_state
    print(); print()

START  s  t  a  ʊ  v  ɪ  m  ɪ  ʃ  ə  n  h  ɝ  d  STOP

START  d  ʒ  ə  t  ə  n  d  ʒ  ɛ  d  STOP

START  b  ɝ  d  ʒ  ə  l  ʌ  n  ə  z  STOP

START  t  ə  n  ɛ  f  k  ə  l  ɪ  s  ɛ  f  ə  n  ɪ  d  e  ɪ  l  i  STOP

START  ʃ  p  ɪ  k  ə  STOP

START  p  æ  t  ɹ  i  STOP

START  ɡ  ə  STOP

START  ə  l  ɬ  fː  ɝ  l  æ  k  STOP

START  s  o  ʊ  STOP

START  s  l  STOP

START  ɡ  o  ʊ  ɛ  ʒ  ɪ  t  e  ɪ  t  STOP

START  k  j  ʌ  n  ə  ɹ  ɪ  k  STOP

START  d  STOP

START  b  ɹ  ɑ  z  e  ɪ  ʃ  ə  t  ɑ  n  t  h  ɪ  t  s  ɪ  z  ɪ  ŋ  z  ɪ  k  ə  m  ɪ  n  ɒ  n  ɒ  ɹ  STOP

START  æ  k  ɪ  ɡ  ɹ  θ  ə  k  s  t  ə  b  ɹ  ɛ̝  ʊ̯  l  ɪ  k  e  ɪ  ŋ  STOP

START  k  STOP

START  ɹ  i  STOP

START  h  ɑ  ɹ  t  ʃ  STOP

START  e  ɪ  z  STOP

START  s  t  e  ɪ  t  l  STOP



OK, that seemed to more or less work.  We get some reasonable-looking outputs, and also some that look a little strange.

Now, just to highlight how natural some of the stranger-looking outputs we just saw were, we'll try the same thing, but flipping the roles of $p_1$ and $p_2$ in the counts.  That is, when we see the bigram $p_1 p_2$, we increment the count for $p_2 p_1$.  

What does this do?  It rewards or strengthens transitions that tend to show up in the **opposite order** from that actually encountered.  

What sort of output should we predict?

**[ YOUR ANSWER GOES HERE. ]**

**1. Each output word tend to be very long, often with multiple encounters with "START" in the middle before getting to "STOP."**

**2. The combination of phonemes in each output word will sound more natural when read backwards then read normally.**

In [12]:
# generate transition probabilities from counts again, but this time with p1 and p2 swapped, 
# as described above.  this will be a lightly but strategically edited variant of a code box
# above.
tp_swapped = dict()  # transition probs
for p1 in phonemes:
    d = dict()
    # YOUR CODE GOES HERE.
    p1_total = 0
    for key in ptc.keys():
        p1_total += ptc[key] if key[1] == p1 else 0
    for p2 in phonemes:
        d[p2] = ptc[(p2, p1)] / p1_total
    
    tp_swapped[p1] = d

In [13]:
# now generate 20 pseudo-words with these swapped transition probabilities.
for i in range(20):
    # for each pseudo-word, we start by setting state to 'START'.
    state = 'START'
    print(state, end='')   # print state, but no newline.

    # now keep repeating this until you arrive at the 'STOP' state
    while (state != 'STOP'):
        next_state_options = list(tp_swapped[state].keys())
        next_state_probs = list(tp_swapped[state].values())
        new_state = np.random.choice(next_state_options, p=next_state_probs)
        print(" ", new_state, end='')
        state = new_state
    print(); print()

START  lʲ  fː  q  õ  z  ɑː  z  æ  START  ɛ͡ɹ  iː  ʒ  əː  ʌː  ɪ̪  ɲ  f  START  e͜ɚ  äː  uː  s  START  ɘ  k  ɪ  START  ʉ  ʌ̈  äː  ɱ  ɛ̝  ɑ̃  ɲ  ʃʰ  ē  bʲ  o͡ʊ  ɵ  ʃː  ú  ɾ  t̬  e͡ɪ  ɪː  iː  n  ʊ  o  z  ɹ  ə  n  ɪ  k  START  dʲ  ɒː  æ̃  ʉː  ɹ̩  mː  ä  ô  k̚  nː  dʰ  ɭ  iː  ɹ  i  d  n  START  ɔː  k  ɛ  ʒ  ɪ̈  ɐː  t  i  t  s  START  t͡ʃ̩̩  t̬ᵊ  ɡː  ɔ  n  æ  k  START  ɻ  ʌː  ð  ɪ  l  ɡ  START  ʌ  s  n  æ  START  y  a  START  ɹ  b  æ  p  ɑ  d  START  ʊ̆  ʔ  ɹː  q  n  ɪ  e  l  k  START  bʲ  j  n  æ  ɹ  t  k  æ  k  START  ú  h  e͡ɪ  tᵊ  ɬ  ɪ  s  æ  h  ʊ  o  d͡ʒ  ɪ  e  ɹ  ə  d  æ  ɡ  START  x  ĭ̥  ɱ  oˀ  ʒ  d  ɪ  a  k  START  t͡s  ɡ  z  m  ɛ  n  æ  ɹ  k  START  ɘ  ɖ  ɛ  START  nˡ  uː  t  ə  s  START  bᵊ  t͡ʃʰ  ç  oː  nˡ  ʊ̯  ɻ  ɪ̈  ɡ  ɑ  p  m  ʊ  ə  START  fː  ɑ  p  m  ʃ  æ  START  õ  tː  y  j  t  ɛ  START  ʀ  ɜ  ʒ  d  n  ɪ  w  START  ʊ̩  ʌ̈  x  ɖ  m̩  ɛˑ  ʉː  ú  t͡ʃ̩̩  ʊ̪  e͜ɚ  ɝː  p  ɑ  b  ə  START  ːs  ʃ  ɪ  m  START  oː  ɵ̞  ɡː  z  bʲ  ɛː  h  t  s  k  ʊ  f  ɑ  p  START  eː  l͡s  ɪ̯  a  ɹ  ɛ 

START  ē  ɚ  p  ɹ  ɔ  t  ɛ  k  ɪ  e  l  ɛ  l  ə  ɡ  ʊ  a  d  ɚ  t  ɝ  θ  æ  START  l͡s  e  t  s  START  t  ɪ  ɹ  p  ɛ  l  ɛ  t  k  l  b  ə  r  ɡ  ɪ  START  ɡː  ɑː  p  START  œ  ɪ  e  ʃ  ɪ  ɹ  ɑ  m  ʌ  m  ɪ  e  l  ə  t  ə  n  ʊ  t  ə  n  ɨ  ɛ͡ɪ  ɾ̃  t  ɪ  w  ɡ  t  START  ɡː  ʊ̪  ɪ̯  ʉː  ə̆  ɪˑ  ɛˑ  əʳ  ɒ̃  n  ɒ  w  START  ɾ̃  ʊ̪  əʳ  ɝː  fː  ↓  l̩  t  ʊ  o  l  START  t͡s  p̬  ʈ  ʉː  ɛˑ  u  j  k  ɪ  e  l  ə  l  START  ɐː  p  æ  START  ŋ  ɪ  d  ɪ  ɹ  ɑ  ɛ͡ɪ  q  ɒ  p  START  lʲ  n  ɨ  h  START  d  START  ə̆  ɛ̝  p̬  ɪ̯  e  l  ʊ  f  START  ɑ  m  ʌ  ɪ̯  e  v  æ  h  t  START  dʰ  t̬  əʳ  õ  əː  ú  ʊ  p  ʊ  o  ɹ  ə  p  s  t  s  ɪ  ɔ  l  f  START  ð  r  ə  ɪ  a  h  START  ɝ  p  s  START  χ  START  o̞  ɫ̩  b  START  θ  START  ç  ɲ  ʊ̯  ɫ  ɡː  l͡s  e  t  k  uː  ʃ  ɑː  k  ɛ  ɹ  k  ɪ  ɡ  ŭ̥  ɡ  ŋ  æ  l  ʊ  f  l  ə  l  p  m  START  œ  ɹʷ  ɚ  i  t  ɪ  h  START  t͡ɕ  t͡ʃ̩̩  ɪ̈  ɹ  k  ɪ  ɹ  ə  ʃ  START  ɐ  m  ɪ  v  ɪ  e  z  ɪ  e  w  START  t͡ʃ̩̩  ø  kᵊ  t͡ʃ̩̩  ʉː  j  b  START  ɘ  əː  θ  ɒ  k  ŋ  ɑː  t 

START  ɵ̞  ɒ̃  ä  ʊ̯  a  ɹ  ɛ  k  ə  w  s  ɪ  ɹ  ɑː  s  START  ɔ͡ɪ  m̩ː  bᵊ  ɪˑ  ɪʲ  ɝː  t  n  ɪ  a  START  eː  m̩ː  tː  ə  v  START  ɖ  ɑː  h  START  ʃʰ  ɭ  ä  ʊ̩  ɚː  ʉː  lʲ  ʀ  ɪ̈  ð  ʒ  d  START  ɦ  ɐː  t͡s  ʊ̩  ɕː  ɛː  ɡ  ɪ  ɹ  ə  v  ɪ  m  ə  v  ɪ  e  k  iː  t  ɛ  START  ɜ̃  ɑː  START  ɻ  ð  ɭ  ú  eː  əʳ  ɡː  ɣ  bʲ  tʰ  ɛ̝  ɜː  z  iː  w  START  ə̆  ɛˑ  ʃʰ  l̩  z  ŋ  æ  l  k  ɪ  n  ɹ  ɡ  æ  m  START  ɾ̃  m̩ː  ɒː  ʌ̈  o̞  t͡ʃ  ɪ  e  d  ə  ɹ  ə  s  t  s  ə  ɹ  ɔ  p  START  ɪ̪  ɡ  START  ɡ  ɪ  e  ɹ  k  START  ɫ̩  ɕː  ɒː  t̬ᵊ  tʰ  ɪ̈  i  d  ə  z  ɪ  l  ə  ʃ  START  nˡ  ɔː  START  ɑ  l  ɚ  ɛ  ɹ  ɑ  s  n  ɪ  t  k  START  ʃʰ  ːs  m̩  ɹʷ  ʊ̩  tʰ  t͡ɕ  s  iː  w  b  ə  ɪ  f  ɪ  ɹ  ə  START  ɪ̈  ɕː  ɝː  f  START  æː  r̩  t͡s  o̞  ɨ  t  n  æ  b  ə  ɹ  k  START  l̩  b  ə  θ  ɪ  START  ɵ̞  k  START  ɕː  ɦ  STOP

START  ð  START  uː  n  ɪ  ɔ  t  s  ʊ  o  k  START  ʉ  ɭ  v  ə  t  ɪ  ɹ  ə  START  ɹ  ɡ  START  ɖ  p̬  bʲ  tʰ  ç  ʁ  sː  ɬ  mː  k̚  ɑ  k  ə  START  ʈ  ʊ̂  q  ɜ  p  ɛ  s  n  æ  p  START  

# Observations

- Write your general observations about the results here.  A few short sentences would be fine.

It seems that phoneme transition probabilities are generally good at generating English pseudo-words, i.e., words that comply with the "rules" of English pronunciation, except for several single-phoneme ones. However, it is reasonable to speculate that transition probability alone probably does not generate well to words (i.e., generating sentences that comply with rules of English grammar). On the one hand, it may generate sentences that are statistically probable but ungrammatical, e.g., "I like to the cafeteria;" on the other hand, it cannot generate sentences that are grammatically but meaningless, such as the good o old "colorless green ideas sleep furiously." Therefore, the behaviorist view of a "sequential" language has its limitations. Sentential grammar is more than a sequence probability; that is where other views of language, such as the generative grammar of Chomsky, come in.

# Extensions (optional)

- Do the same thing for other languages.  See [here](https://github.com/kylebgorman/wikipron/tree/master/data) for analogous WikiPron data from other languages, in the same format as the data we've used here.
- Do the same thing for words rather than phonemes.  You could download a book in plain text from [Project Gutenberg](https://www.gutenberg.org/ebooks/search/?sort_order=downloads), and proceed from there.